# Extração de dados partidos politicos desde 1996 ate 2022

In [ ]:
import requests

from bs4 import BeautifulSoup as bs

import pandas as pd

In [2]:
years = ['2022', '2019', '2015', '2011', '2009', '2005', '2002', '1999', '1995']
tags = ['Resultados_por_círculo_eleitoral', 'Tabela_de_resultados_por_círculos_eleitorais', 'Tabela_de_resultados']

In [5]:
df = pd.DataFrame(columns=['year','city', 'political_party', 'votes'])

for year in years:
    print(year)
    url = f'https://pt.wikipedia.org/wiki/Elei%C3%A7%C3%B5es_legislativas_portuguesas_de_{year}'
    request = requests.get(url)
    soup = bs(request.text, 'html.parser')
    
    for tag in tags:
        s = soup.find("span", id=tag)
        if s:
            try:
                for table_index in range(len(s.find_all_next('table'))):
                    for idx in range(len(s.find_all_next('table')[table_index].find_all('td'))):
                        if s.find_all_next('table')[table_index].find_all('td')[idx].find('a'):
                            
                            city = s.find_all_next('table')[table_index].find_previous_sibling('h3').find('span', attrs={'class': 'mw-headline'}).text 
                            partido = s.find_all_next('table')[table_index].find_all('td')[idx].find('a').text
                            votos = s.find_all_next('table')[table_index].find_all('td')[idx].find_next('td').text
                            temp = {'year': year, 'city' : city, 'political_party' : partido, 'votes': float(votos.replace('\n', '').replace(' ', ''))}
                            df = pd.concat([df, pd.DataFrame(temp, index=[idx])], ignore_index=True)
            except:
                continue

2022
2019
2015
2011
2009
2005
2002
1999
1995


In [6]:
df.year.value_counts()

2022    174
2019    161
2015    131
2011    131
2009    123
2005    114
2002    112
1995     96
Name: year, dtype: int64

In [9]:
df.head()

,year,city,political_party,votes
0,2022,Açores,Partido Socialista,36025.0
1,2022,Açores,Aliança Democrática,28520.0
2,2022,Açores,CHEGA,4986.0
3,2022,Açores,Bloco de Esquerda,3589.0
4,2022,Açores,Iniciativa Liberal,3454.0


In [10]:
df.groupby(['city', 'political_party']).sum().head(50)

year  \
city   political_party                                                                     
Aveiro Bloco de Esquerda                                    2022201920152011200920052002   
       CDS – Partido Popular                                            2022201920112009   
       CDU – Coligação Democrática Unitária                                         2019   
       CHEGA                                                                        2022   
       Coligação Democrática Unitária                       2022201520112009200520021995   
       Iniciativa Liberal                                                       20222019   
       PESSOAS–ANIMAIS–NATUREZA                                                     2019   
       Partido Democrático Republicano                                              2015   
       Partido Popular                                                      200520021995   
       Partido Social Democrata                             2022201920112009200520021995   
       Partido Socialista                               20222019201520112009200520021995   
       Pessoas–Animais–Natureza                                                     2022   
       Portugal à Frente                                                            2015   
Açores Aliança Açores                                                               2015   
       Aliança Democrática                                                          2022   
       Bloco de Esquerda                                    2022201920152011200920052002   
       CDS – Partido Popular                                                201920112009   
       CDU – Coligação Democrática Unitária                                         2019   
       CHEGA                                                                        2022   
       Coligação Democrática Unitária                       2022201520112009200520021995   
       Iniciativa Liberal                                                           2022   
       PESSOAS–ANIMAIS–NATUREZA                                                     2019   
       Partido Popular                                                      200520021995   
       Partido Social Democrata                             2019201520112009200520021995   
       Partido Socialista                               20222019201520112009200520021995   
       Pessoas–Animais–Natureza                                                     2022   
Beja   Bloco de Esquerda                                    2022201920152011200920052002   
       CDS – Partido Popular                                                201920112009   
       CDU – Coligação Democrática Unitária                                         2019   
       CHEGA                                                                    20222019   
       Coligação Democrática Unitária                       2022201520112009200520021995   
       Iniciativa Liberal                                                           2022   
       PCTP/MRPP                                                201520112009200520021995   
       PESSOAS–ANIMAIS–NATUREZA                                                     2019   
       Partido Comunista dos Trabalhadores Portugueses                              2019   
       Partido Popular                                                      200520021995   
       Partido Social Democrata                             2022201920112009200520021995   
       Partido Socialista                               20222019201520112009200520021995   
       Portugal à Frente                                                            2015   
       União Democrática Popular                                                    1995   
Braga  Bloco de Esquerda                                    2022201920152011200920052002   
       CDS – Partido Popular                                            2022201920112009   
       CDU – Coligação Democrática U